In [57]:
import pandas as pd, numpy as np, json, os
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
m=json.loads(open('json/member_timelines.json','r').read())

In [3]:
df=pd.DataFrame(m).groupby(['Name','Csoportok']).first()

In [4]:
df.head(8)

Csoportok0  \
Name         Csoportok                                                                                      
ALBERT Álmos ⏺️ Európa                                                                          ⏺️ Európa   
             🇨🇺 Kuba                                                                              🇨🇺 Kuba   
             🇩🇿 Algéria                                                                        🇩🇿 Algéria   
             🇫🇷 Franciaország                                                            🇫🇷 Franciaország   
             🇲🇦 Marokkó                                                                        🇲🇦 Marokkó   
             🇷🇴 Kovászna                                                                      🇷🇴 Kovászna   
             🌷 RMDSZ - Romániai Magyar Demokrata Szövetség  🌷 RMDSZ - Romániai Magyar Demokrata Szövetség   
             🔀 Állami ügyek                                                                🔀 Állami ügyek   

                                                                                               Csoportok1  \
Name         Csoportok                                                                                      
ALBERT Álmos ⏺️ Európa                                                                          ⏺️ Európa   
             🇨🇺 Kuba                                                                              🇨🇺 Kuba   
             🇩🇿 Algéria                                                                        🇩🇿 Algéria   
             🇫🇷 Franciaország                                                            🇫🇷 Franciaország   
             🇲🇦 Marokkó                                                                        🇲🇦 Marokkó   
             🇷🇴 Kovászna                                                                      🇷🇴 Kovászna   
             🌷 RMDSZ - Romániai Magyar Demokrata Szövetség  🌷 RMDSZ - Romániai Magyar Demokrata Szövetség   
             🔀 Állami ügyek                                                                🔀 Állami ügyek   

                                                                  Simple  \
Name         Csoportok                                                     
ALBERT Álmos ⏺️ Európa                                      Albert Almos   
             🇨🇺 Kuba                                        Albert Almos   
             🇩🇿 Algéria                                     Albert Almos   
             🇫🇷 Franciaország                               Albert Almos   
             🇲🇦 Marokkó                                     Albert Almos   
             🇷🇴 Kovászna                                    Albert Almos   
             🌷 RMDSZ - Romániai Magyar Demokrata Szövetség  Albert Almos   
             🔀 Állami ügyek                                 Albert Almos   

                                                                Típus  \
Name         Csoportok                                                  
ALBERT Álmos ⏺️ Európa                                      Bizottság   
             🇨🇺 Kuba                                           Ország   
             🇩🇿 Algéria                                        Ország   
             🇫🇷 Franciaország                                  Ország   
             🇲🇦 Marokkó                                        Ország   
             🇷🇴 Kovászna                                        Megye   
             🌷 RMDSZ - Romániai Magyar Demokrata Szövetség       Párt   
             🔀 Állami ügyek                                 Bizottság   

                                                           duration  \
Name         Csoportok                                                
ALBERT Álmos ⏺️ Európa                                         1462   
             🇨🇺 Kuba                                           1462   
             🇩🇿 Algéria                                        1462   
             🇫🇷 Franciaország                                  129

In [47]:
tipusok={'Bizottság':['Bizottság'], 
         'Ország':['Ország'], 
         'Megye':['Párt','Bizottság','Ország'], 
         'Párt':['Bizottság','Ország']}
edges={'Total':{'Total':{}}}
members_set=df.reset_index()['Name'].unique()
for iname,name in enumerate(members_set[:]):
    if iname%100==0: print(iname/len(members_set)*100,'%')
    sdf=df.loc[name]
    csoportok=sdf.index
    tipus=sdf['Típus'].values
    start=sdf['start'].astype('datetime64').dt.date.values
    end=sdf['end'].astype('datetime64').dt.date.values
    for t in pd.date_range(min(sdf['start']),max(sdf['end'])):
        td=t.date()
        for i in range(len(csoportok)):
            csop=csoportok[i]
            tip=tipus[i]
            if tip in tipusok:
                if start[i]<td<end[i]:
                    for j in range(len(csoportok)):
                        if i!=j:
                            csop2=csoportok[j]
                            tip2=tipus[j]
                            if tip2 in tipusok[tip]:
                                if start[j]<td<end[j]:
                                    nset=tuple(np.sort([csop,csop2]))

                                    if tip not in edges:edges[tip]={}
                                    if tip2 not in edges[tip]:edges[tip][tip2]={}
                                    if nset not in edges[tip][tip2]:edges[tip][tip2][nset]=0
                                    edges[tip][tip2][nset]+=1

                                    if nset not in edges:edges['Total']['Total'][nset]=0
                                    edges['Total']['Total'][nset]+=1

0.0 %
3.5855145213338115 %
7.171029042667623 %
10.756543564001435 %
14.342058085335246 %
17.927572606669056 %
21.51308712800287 %
25.098601649336683 %
28.684116170670492 %
32.269630692004306 %
35.85514521333811 %
39.440659734671925 %
43.02617425600574 %
46.61168877733955 %
50.197203298673365 %
53.78271782000718 %
57.368232341340985 %
60.9537468626748 %
64.53926138400861 %
68.12477590534242 %
71.71029042667622 %
75.29580494801003 %
78.88131946934385 %
82.46683399067766 %
86.05234851201148 %
89.63786303334528 %
93.2233775546791 %
96.80889207601291 %


In [139]:
network={}
for tip in edges:
    for tip2 in edges[tip]:
        #create graph
        edgelist=[]
        for e in edges[tip][tip2]:
            edgelist.append((e[0],e[1],edges[tip][tip2][e]))
        G=nx.Graph()
        G.add_weighted_edges_from(edgelist)

        #nodes
        pr = nx.pagerank(G,weight='weight')
        nodes={i:{'id':i,'pr':pr[i]/np.median(list(pr.values()))} for i in pr}

        dc=nx.degree_centrality(G)
        for name in dc:
            nodes[name]['dc']=dc[name]/np.median(list(dc.values()))

        from networkx.algorithms.community import greedy_modularity_communities
        communities = list(greedy_modularity_communities(G,weight='weight'))
        for i,names in enumerate(sorted(map(sorted, communities))):
            for name in names:
                nodes[name]['modularity']='c'+str(i)
                
        from networkx.algorithms import community
        communities_generator = community.girvan_newman(G)
        for i in range(3):
            communities = next(communities_generator)
        for i,names in enumerate(sorted(map(sorted, communities))):
            for name in names:
                nodes[name]['community']='c'+str(i)
                
#         from networkx.algorithms.community import k_clique_communities
#         communities = list(k_clique_communities(G, 7))
#         for i,names in enumerate(sorted(map(sorted, communities))):
#             for name in names:
#                 nodes[name]['kclique']='c'+str(i)

        #edges
        ebc=nx.edge_betweenness_centrality(G,weight='weight') #weight
        links=[]
        for e in G.edges(data=True):
            links.append({'source':e[0],'target':e[1],
                          'value':e[2]['weight'],
                          'ebc':ebc[(e[0],e[1])]
                         })
            
        #save 
        if tip not in network:network[tip]={}
        network[tip][tip2]={'nodes':nodes,'links':links}
                                    
        print(tip,tip2,len(nodes),'nodes and',len(links),'edges')

Total Total 195 nodes and 9279 edges
Bizottság Bizottság 14 nodes and 91 edges
Ország Ország 99 nodes and 3269 edges
Megye Bizottság 57 nodes and 568 edges
Megye Ország 140 nodes and 2754 edges
Megye Párt 82 nodes and 668 edges
Párt Bizottság 52 nodes and 343 edges
Párt Ország 133 nodes and 1586 edges


In [140]:
open('json/plotter_csoportok.json','w').write(json.dumps(network))

2871411